In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 10 messages


[None, None, None, None, None, None, None]

# ATMCS

In [22]:
await asyncio.sleep(5.)
print("Done")
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 82 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 81 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 77 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 77 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 82 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 77 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 47 elements
AckCmdReader(ATMCS, 0, ackcmd) f

Done


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

# ATPtg

In [18]:
await salobj.set_summary_state(atptg, salobj.State.STANDBY)

RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 89 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 89 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 68 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 89 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 90 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 68 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 90 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind; read 90 messages
RemoteTelemetry(ATPtg, 0, mountStatus)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [19]:
await asyncio.sleep(5.)
print("Done")
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

RemoteTelemetry(ATDome, 1, position) falling behind; read 12 messages


Done


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [7]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [14]:
await atptg.cmd_azElTarget.set_start(azDegs=-126, elDegs=62.)

falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 85 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 94 elements
falling behind; queue contains 93 elements
falling behind; queue contains 92 elements


In [20]:
ra = Angle("22:42:40.05027", unit=u.hour)
dec = Angle("-46:53:04.4752", unit=u.deg)
target_name="Bet Gru"
radec = ICRS(ra, dec)

In [21]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 12 messages
RemoteEvent(ATPtg, 0, mountDataWarning) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 12 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 59 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 26 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 12 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 12 messages
falling behind; queue contains 25 elements
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 12 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 60 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 60 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 12 mess

58739.08670230144
slew...


In [10]:
await atptg.cmd_stopTracking.start(timeout=10)

RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 38 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 38 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 38 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 29 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 38 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 29 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 38 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-30., 
                                     az=0., 
                                         num=0)

In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
grid_alt = np.zeros(len(dalt_grid)*len(daz_grid))
grid_az = np.zeros(len(dalt_grid)*len(daz_grid))

for i in range(len(daz_grid)):
    grid_alt[i*len(dalt_grid):(i+1)*len(dalt_grid)] += dalt_grid[::(-1)**i]
    grid_az[i*len(dalt_grid):(i+1)*len(dalt_grid)] += daz_grid[i]

In [ ]:
plt.plot(grid_alt, grid_az, ':')
plt.plot(grid_alt, grid_az, '|')
plt.plot(grid_alt[0], grid_az[0], 'bo')
plt.plot(grid_alt[-1], grid_az[-1], 'go')

In [ ]:
start_from=0

In [ ]:
print(f"Grid has {len(grid_alt)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

while i < len(grid_alt):
    
    print(f"GRID[{i}]: {grid_alt[i]} x {grid_az[i]}")

    await atptg.cmd_offsetAzEl.set_start(el=grid_alt[i]*60.*60., 
                                         az=grid_az[i]*60*60, 
                                         num=0)
    
    ret_val = await wait_center()

    if ret_val == 0:
        break
        
    i += ret_val
    
    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=0., 
                                     az=-20, 
                                     num=1)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-15., 
                                     az=0, 
                                         num=1)

In [ ]:
60-61

In [ ]:
await asyncio.sleep(5)
off = await atptg.tel_guidingAndOffsets.next(flush=True, timeout=5)

In [ ]:
print(off)